In [1]:
import utils
from utils import plots

Using TensorFlow backend.
/home/dbashir/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import numpy as np
from glob import glob
import os
import cv2

In [3]:
path = 'pdf/model_testing/'

In [4]:
train_path = os.path.join(path,'train')
valid_path = os.path.join(path,'valid')
test_path = os.path.join(path,'test')

In [5]:
from utils import *

In [6]:
batch_size = 2
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 60 images belonging to 30 classes.
Found 10 images belonging to 5 classes.


In [7]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 60 images belonging to 30 classes.
Found 10 images belonging to 5 classes.
Found 0 images belonging to 0 classes.


In [8]:
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [9]:
# x_train = trn_classes
# x_test = val_classes

In [10]:
#x_train = []

In [11]:
# #i = 0
# for root, dirnames, filenames in os.walk(train_path):
#     for dirname in dirnames:
#         print(dirname)
#         path = os.path.join(train_path, dirname)
#         for fileWanted in os.listdir(path):
#             if '.jpg' in fileWanted:
#                 image = os.path.join(path,fileWanted)
#                 img = cv2.imread(image)
#                 img = img.reshape(800,600,1)
#                 np.append(x_train,img)
#                 #i += 1

In [12]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [13]:
X_train = datagen.flow_from_directory(train_path, target_size=(800,600), color_mode='grayscale', batch_size=60, class_mode='categorical')
X_valid = datagen.flow_from_directory(valid_path, target_size=(800,600), color_mode='grayscale', batch_size=60, class_mode='categorical')
#X_test = datagen.flow_from_directory(test_path, target_size=(800,600), batch_size=60, class_mode='categorical')

Found 60 images belonging to 30 classes.
Found 10 images belonging to 5 classes.


In [14]:
x_train, y_train = X_train.next()

/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [15]:
x_test,y_test = X_valid.next()

/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [16]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 800, 600, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 800, 600, 1))  # adapt this if using `channels_first` image data format

np.save('x_train', x_train)

In [17]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

In [18]:
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [19]:
def get_model():
    input_img = Input(shape=(800, 600, 1))  # adapt this if using `channels_first` image data format
    x = Conv2D(16, (3, 3), activation='relu', border_mode='same')(input_img)
    x = MaxPooling2D((2, 2), border_mode='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', border_mode='same')(x)
    x = MaxPooling2D((2, 2), border_mode='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', border_mode='same')(x)
    encoded = MaxPooling2D((2, 2), border_mode='same', name='encoder')(x)

    # at this point the representation is (4, 4, 8) i.e. 128-dimensional

    x = Conv2D(8, (3, 3), activation='relu', border_mode='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', border_mode='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', border_mode='same')(x)

    autoencoder = Model(input_img, decoded)
    return autoencoder.summary()
    
#     autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

#     autoencoder.fit(x_train_noisy, x_train,
#                     epochs=20,
#                     batch_size=12,
#                     shuffle=True,
#                     validation_data=(x_test_noisy, x_test),
#                     callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])

#     autoencoder.save('autoencoder.h5')

In [20]:
get_model()

/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", padding="same")`
  app.launch_new_instance()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 800, 600, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 600, 1)        115216    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 300, 1)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 300, 1)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 150, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 150, 1)         584       
_________________________________________________________________
encoder (MaxPooling2D)       (None, 8, 75, 1)          0         
__________

/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), padding="same")`
/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", padding="same")`
/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), padding="same")`
/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", padding="same")`
/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), name="encoder", padding="same")`
/home/dba

In [21]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [32]:
def train_model():
    input_img = Input(shape=(800, 600, 1))  # adapt this if using `channels_first` image data format
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), border_mode='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), border_mode='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same', name='encoder')(x)

    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    
    print(autoencoder.summary())
    #SVG(model_to_dot(autoencoder,show_shapes=True).create(prog='dot', format='svg'))

    autoencoder.fit(x_train_noisy, x_train,
                    epochs=20,
                    batch_size=12,
                    shuffle=True,
                    validation_data=(x_test_noisy, x_test),
                    callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])

    autoencoder.save('autoencoder.h5')

In [33]:
train_model()

/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), padding="same")`
/home/dbashir/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), padding="same")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 800, 600, 1)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 800, 600, 16)      160       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 400, 300, 16)      0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 400, 300, 8)       1160      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 200, 150, 8)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 200, 150, 8)       584       
_________________________________________________________________
encoder (MaxPooling2D)       (None, 100, 75, 8)        0         
__________